In [26]:
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval,
    BertForSequenceClassification)
import torch
import numpy as np

In [1]:
import pandas as pd

/home/uma_roy_us_gmail_com/.local/share/virtualenvs/transformers-_CUu7l8n/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [21]:
df = pd.read_csv('../data_dir/doordash_test.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 189: invalid start byte

In [13]:
df.replace(r'\n', '', regex = True, inplace = True)

In [20]:
df.dropna()

,Title,Description,Manual Cuisine,Manual Dish Type,manual cuisine,manual dish type
0,BBQ Pork Bao,Char-siu roasted pork belly,Chinese,Other,chinese,other
1,Seafood Gyoza,"Spinach skin, crab, scallop, shrimp, chilli, b...",Japanese,Other,japanese,other
2,Impossible MeltBurger,Our signature meltburger made with a plant-bas...,American,Burger,american,burger
3,Steak & Cheese,"Grilled steak, provolone, melted cheddar, cara...",American,Sandwich,american,sandwich
4,Mission,"Romaine hearts, cabbage, green chili chicken, ...",Mexican,Salad,mexican,salad
...,...,...,...,...,...,...
67,Tom Yum Noodle Soup,"Hot & spicy noodle soup with ground chicken, s...",thai,soup,thai,soup
69,Super Food,"Garden salad mix, quinoa, diced tomatoes, cucu...",other,salad,other,salad
71,Southwest BBQ Chicken,"Chargrilled chicken breast atop garden greens,...",other,salad,other,salad
73,Santa Barbara Cobb,"Crisp shredded iceberg and Romaine lettuce, di...",other,salad,other,salad


In [16]:
titles = df.Title

In [57]:
true_cuisines = df['manual cuisine']

In [66]:
true_cuisines_list = list(true_cuisines[titles.dropna().index].values)

In [67]:
true_cuisines_list

['chinese',
 'japanese',
 'american',
 'american',
 'mexican',
 'other',
 'other',
 'thai',
 'other',
 'indian',
 'indian',
 'indian',
 'mexican',
 'american',
 'thai ',
 'thai',
 'thai ',
 'thai ',
 'thai ',
 'indian ',
 'italian',
 'indian ',
 'italian',
 'indian',
 'italian',
 'indian',
 'italian',
 'japanese',
 'japanese',
 'japanese',
 'japanese',
 'italian',
 'japanese',
 'mexican',
 'japanese',
 'other ',
 'mexican',
 'mexican',
 'american',
 'mexican',
 'other',
 'chinese',
 'other',
 'chinese',
 'other',
 nan,
 'other',
 'chinese',
 'chinese',
 'other',
 'american',
 'other',
 'other',
 'american',
 'american',
 'american',
 'american',
 'american',
 'american',
 'american',
 'japanese',
 'other',
 'italian',
 'other',
 'thai',
 'other',
 'american',
 'other',
 'other',
 'japanese',
 'italian',
 'italian']

In [61]:
titles.dropna().index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 49, 50, 51, 52,
            53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 69, 70, 71,
            73, 74, 75, 76],
           dtype='int64')

In [19]:
titles.values

array(['BBQ Pork Bao', 'Seafood Gyoza', 'Impossible MeltBurger',
       'Steak & Cheese', 'Mission', 'Pacifica Grain Bowl',
       'Tea Leaf Salad-Laphet', 'Spicy Noodles', 'Burmese Tomato Chutney',
       'Tikka Masala Burrito', 'Naughty Naan', 'Samosas',
       'Super Meat Nachos', 'Jumbo Wings (10-pc Regular Pack)',
       'Pad-King Rice Plate (Lunch)',
       'Tom Yum Goong (Spicy Shrimp Lemongrass Soup)',
       'Green Papaya Salad (Som Tum Thai)', 'Pad Woon Sen',
       'Thai Iced Coffee', 'Malabar Parota', 'The Meats',
       'Paneer Bhurjidosa', 'Tagliatelle', 'Vegetable Briyani',
       'Meat Ravioli', 'Achar Gosht', 'Panna Cotta', '1272 Volcano Roll',
       'Rainbow Roll', '1141 Miso Soup', 'Goma Tuna Tataki',
       'Caprese Salad', 'Pork Gyoza', 'Carne Asada Burrito', 'o toro',
       'Turkey Breast Footlong Regular Sub', 'Guacamole, Chips & Salsa',
       'Made-To-Order Guacamole', 'Buffalo Chicken Sandwich',
       'Just Cheese Quesadilla', '7 Vegetable Tagine with Quino

In [23]:
titles.values

array(['BBQ Pork Bao', 'Seafood Gyoza', 'Impossible MeltBurger',
       'Steak & Cheese', 'Mission', 'Pacifica Grain Bowl',
       'Tea Leaf Salad-Laphet', 'Spicy Noodles', 'Burmese Tomato Chutney',
       'Tikka Masala Burrito', 'Naughty Naan', 'Samosas',
       'Super Meat Nachos', 'Jumbo Wings (10-pc Regular Pack)',
       'Pad-King Rice Plate (Lunch)',
       'Tom Yum Goong (Spicy Shrimp Lemongrass Soup)',
       'Green Papaya Salad (Som Tum Thai)', 'Pad Woon Sen',
       'Thai Iced Coffee', 'Malabar Parota', 'The Meats',
       'Paneer Bhurjidosa', 'Tagliatelle', 'Vegetable Briyani',
       'Meat Ravioli', 'Achar Gosht', 'Panna Cotta', '1272 Volcano Roll',
       'Rainbow Roll', '1141 Miso Soup', 'Goma Tuna Tataki',
       'Caprese Salad', 'Pork Gyoza', 'Carne Asada Burrito', 'o toro',
       'Turkey Breast Footlong Regular Sub', 'Guacamole, Chips & Salsa',
       'Made-To-Order Guacamole', 'Buffalo Chicken Sandwich',
       'Just Cheese Quesadilla', '7 Vegetable Tagine with Quino

In [11]:
# import csv
# with open('../data_dir/doordash_test.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=',')
#     for row in spamreader:
#         print(row)

['Title', 'Description', 'Manual Cuisine', 'Manual Dish Type', 'manual cuisine', 'manual dish type']
['\nBBQ Pork Bao\n', 'Char-siu roasted pork belly', 'Chinese', 'Other', 'chinese', 'other']
['\nSeafood Gyoza', 'Spinach skin, crab, scallop, shrimp, chilli, butter sauce', 'Japanese', 'Other', 'japanese', 'other']
['\nImpossible MeltBurger', 'Our signature meltburger made with a plant-based patty that looks, smells and tastes just like beef! The impossible patty is chopped, grilled and filled with loads of melted cheddar, jalapeño and pickle mix (not too spicy, we promise) and melt sauce on a toasted, artisan bun.', 'American', 'Burger', 'american', 'burger']
['\nSteak & Cheese', 'Grilled steak, provolone, melted cheddar, caramelized onions, red peppers.', 'American', 'Sandwich', 'american', 'sandwich']
['\nMission', 'Romaine hearts, cabbage, green chili chicken, guacamole, cotija cheese, pickled spring vegetables, pico de gallo, jicama, toasted pumpkin seeds and tomatillo vinaigrette.

In [27]:
config = AutoConfig.from_pretrained(
    "../output_dirs/classification_test/checkpoint-4500",
)
tokenizer = BertTokenizer.from_pretrained(
    "../output_dirs/classification_test/checkpoint-4500",
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    "../output_dirs/classification_test/checkpoint-4500",
    from_tf=False,
    config=config,
)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [29]:
model = model.to(device)

In [36]:
list(titles.dropna().values)

array(['BBQ Pork Bao', 'Seafood Gyoza', 'Impossible MeltBurger',
       'Steak & Cheese', 'Mission', 'Pacifica Grain Bowl',
       'Tea Leaf Salad-Laphet', 'Spicy Noodles', 'Burmese Tomato Chutney',
       'Tikka Masala Burrito', 'Naughty Naan', 'Samosas',
       'Super Meat Nachos', 'Jumbo Wings (10-pc Regular Pack)',
       'Pad-King Rice Plate (Lunch)',
       'Tom Yum Goong (Spicy Shrimp Lemongrass Soup)',
       'Green Papaya Salad (Som Tum Thai)', 'Pad Woon Sen',
       'Thai Iced Coffee', 'Malabar Parota', 'The Meats',
       'Paneer Bhurjidosa', 'Tagliatelle', 'Vegetable Briyani',
       'Meat Ravioli', 'Achar Gosht', 'Panna Cotta', '1272 Volcano Roll',
       'Rainbow Roll', '1141 Miso Soup', 'Goma Tuna Tataki',
       'Caprese Salad', 'Pork Gyoza', 'Carne Asada Burrito', 'o toro',
       'Turkey Breast Footlong Regular Sub', 'Guacamole, Chips & Salsa',
       'Made-To-Order Guacamole', 'Buffalo Chicken Sandwich',
       'Just Cheese Quesadilla', '7 Vegetable Tagine with Quino

In [33]:
list(titles.values)

['BBQ Pork Bao',
 'Seafood Gyoza',
 'Impossible MeltBurger',
 'Steak & Cheese',
 'Mission',
 'Pacifica Grain Bowl',
 'Tea Leaf Salad-Laphet',
 'Spicy Noodles',
 'Burmese Tomato Chutney',
 'Tikka Masala Burrito',
 'Naughty Naan',
 'Samosas',
 'Super Meat Nachos',
 'Jumbo Wings (10-pc Regular Pack)',
 'Pad-King Rice Plate (Lunch)',
 'Tom Yum Goong (Spicy Shrimp Lemongrass Soup)',
 'Green Papaya Salad (Som Tum Thai)',
 'Pad Woon Sen',
 'Thai Iced Coffee',
 'Malabar Parota',
 'The Meats',
 'Paneer Bhurjidosa',
 'Tagliatelle',
 'Vegetable Briyani',
 'Meat Ravioli',
 'Achar Gosht',
 'Panna Cotta',
 '1272 Volcano Roll',
 'Rainbow Roll',
 '1141 Miso Soup',
 'Goma Tuna Tataki',
 'Caprese Salad',
 'Pork Gyoza',
 'Carne Asada Burrito',
 'o toro',
 'Turkey Breast Footlong Regular Sub',
 'Guacamole, Chips & Salsa',
 'Made-To-Order Guacamole',
 'Buffalo Chicken Sandwich',
 'Just Cheese Quesadilla',
 '7 Vegetable Tagine with Quinoa',
 'Tingly Mapo Tofu',
 'Lamb with Couscous & Vegetables',
 'Classic 

In [37]:
max_seq_length=128
test_input = tokenizer.batch_encode_plus(
    list(titles.dropna().values),
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [38]:
print(test_input)

{'input_ids': [[101, 170, 10457, 11703, 10183, 10174, 18323, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 14931, 21330, 12680, 175, 15594, 10637, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 38199, 10911, 11533, 59692, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
model_inp = dict(test_input)
model_inp['labels'] = [0] * len(test_input['input_ids'])  # dummy labels
# print(model_inp)
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [40]:
len(model_out)

3

In [41]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([72, 30]), torch.Size([72, 768])]


In [45]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits, axis=1)

In [46]:
print(predicted_logits)

[[-1.5127964   2.3691685   4.341873   ... -2.4910378   2.0898101
   2.1645226 ]
 [-2.1572444   2.1177828   5.0628867  ... -1.0259176   1.3089595
   1.1819791 ]
 [-1.8341074   4.6428447  -0.6655135  ...  0.21785729 -2.5338926
  -2.486979  ]
 ...
 [-2.2409916   1.8151826   5.338579   ... -0.75765055  0.8445573
   0.7809026 ]
 [-2.150877    3.090486   -0.03022024 ...  0.09629192 -1.5352896
  -2.2022543 ]
 [-1.0300223   0.9412789   2.5588963  ... -1.4554477   0.45428598
   0.47762674]]


In [47]:
print(predicted_class)

[ 2 18  1 17 18 24 24  2 24 22 15 18 22 17  2  2  2  2 28 24 24 15 17 18
 17 15 17 18 18 18 18 17 18 22 18  1 22 22 24 22 24 19 23  2 14 23 23  2
  2 24 24 19 22  1  1  1  1  1 24 22 18 19 17 24  2 24 17  1 24 18 17  2]


In [50]:
import pickle
cuisines = pickle.load(open('../data_dir/cuisines_ordered_list.pkl', 'rb'))

In [55]:
list_titles = list(titles.dropna().values)

In [72]:
wrong = 0
for i in range(len(list_titles)):
    pred = cuisines[predicted_class[i]]
    true = true_cuisines_list[i]
    title = list_titles[i]
    if pred != true:
        wrong += 1
        print(title, true, pred)
print(wrong/len(list_titles))

BBQ Pork Bao chinese asian
Steak & Cheese american italian
Mission mexican japanese
Spicy Noodles thai asian
Tikka Masala Burrito indian mexican
Samosas indian japanese
Jumbo Wings (10-pc Regular Pack) american italian
Pad-King Rice Plate (Lunch) thai  asian
Tom Yum Goong (Spicy Shrimp Lemongrass Soup) thai asian
Green Papaya Salad (Som Tum Thai) thai  asian
Pad Woon Sen thai  asian
Thai Iced Coffee thai  thai
Malabar Parota indian  other
The Meats italian other
Paneer Bhurjidosa indian  indian
Vegetable Briyani indian japanese
Turkey Breast Footlong Regular Sub other  american
Buffalo Chicken Sandwich american other
Tingly Mapo Tofu chinese korean
Lamb with Couscous & Vegetables other middle_eastern
Classic Chow Mein chinese asian
Lamb & Beef Gyros Plate other greek
Falafel Burger with Fries nan middle_eastern
Hummus other middle_eastern
Wonton Soup chinese asian
Kung Pao Chicken chinese asian
Nashville Looks So Good on You american other
Kimchee Stew other korean
Vegan Caesar other m

In [69]:
[('predicted', 'title', 'actual')] + [(cuisines[p], t, a)  for p, t, a in zip(predicted_class, list_titles, true_cuisines_list)]

[('predicted', 'title', 'actual'),
 ('asian', 'BBQ Pork Bao', 'chinese'),
 ('japanese', 'Seafood Gyoza', 'japanese'),
 ('american', 'Impossible MeltBurger', 'american'),
 ('italian', 'Steak & Cheese', 'american'),
 ('japanese', 'Mission', 'mexican'),
 ('other', 'Pacifica Grain Bowl', 'other'),
 ('other', 'Tea Leaf Salad-Laphet', 'other'),
 ('asian', 'Spicy Noodles', 'thai'),
 ('other', 'Burmese Tomato Chutney', 'other'),
 ('mexican', 'Tikka Masala Burrito', 'indian'),
 ('indian', 'Naughty Naan', 'indian'),
 ('japanese', 'Samosas', 'indian'),
 ('mexican', 'Super Meat Nachos', 'mexican'),
 ('italian', 'Jumbo Wings (10-pc Regular Pack)', 'american'),
 ('asian', 'Pad-King Rice Plate (Lunch)', 'thai '),
 ('asian', 'Tom Yum Goong (Spicy Shrimp Lemongrass Soup)', 'thai'),
 ('asian', 'Green Papaya Salad (Som Tum Thai)', 'thai '),
 ('asian', 'Pad Woon Sen', 'thai '),
 ('thai', 'Thai Iced Coffee', 'thai '),
 ('other', 'Malabar Parota', 'indian '),
 ('other', 'The Meats', 'italian'),
 ('indian', 